# Rule: **add_extra_components**

**Outputs**

- resources/networks/`elec_s{simpl}_{clusters}_ec.nc`

In [ ]:
######################################## Parameters

### Run
name = '04_updated_ALL'
prefix = '1H'

### Network
simpl = ''
clusters = 35

In [ ]:
##### Import packages
import pypsa
import pandas as pd
import cartopy.crs as ccrs
import geopandas as gpd
import matplotlib.pyplot as plt
import yaml
import os 
import sys
import numpy as np
import seaborn as sns


##### Import local functions
sys.path.append(os.path.abspath(os.path.join('..')))
import functions as xp


##### Read params.yaml
with open('../params.yaml', 'r') as configfile:
    params = yaml.safe_load(configfile)


##### Ignore warnings
import warnings
warnings.filterwarnings('ignore', category=UserWarning)


##### Region files
file_regions_onshore = f'regions_onshore_elec_s{simpl}_{clusters}.geojson'
file_regions_offshore = f'regions_offshore_elec_s{simpl}_{clusters}.geojson'
path_regions = f'{params['rootpath']}/resources/{prefix}/{name}/'
gdf_regions_onshore = gpd.read_file(path_regions+file_regions_onshore)
gdf_regions_offshore = gpd.read_file(path_regions+file_regions_offshore)


##### NUTS files (must contain at least columns 'NUTS_ID' and 'geometry')
file_NUTS2 = 'NUTS2_ES.geojson'
file_NUTS3 = 'NUTS3_ES.geojson'
path_NUTS = f'{params['rootpath']}/data_ES/nuts/'
gdf_NUTS2 = gpd.read_file(path_NUTS+file_NUTS2)
gdf_NUTS3 = gpd.read_file(path_NUTS+file_NUTS3)


## `elec_s{simpl}_{clusters}_ec.nc`

Load the network and show its components.

In [ ]:
file = f'elec_s{simpl}_{clusters}_ec.nc'
path = f'{params['rootpath']}/resources/{prefix}/{name}/networks/'

n = pypsa.Network(path+file)

n

Plot the network.

In [ ]:
#################### Parameters
line_widths = 1*n.lines.s_nom / 1e3
link_widths = 1*n.links.p_nom / 1e3



#################### Figure
fig_size = [12,12]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Add network
n.plot(ax=ax, line_widths=line_widths, link_widths=link_widths, bus_sizes=params['bus_sizes'], bus_colors=params['bus_colors'], boundaries=params['boundaries_offshore'])

### Add regions_onshore
xp.map_add_region(ax, gdf_regions_onshore, params['map_add_region'])

### Add regions_offshore
xp.map_add_region(ax, gdf_regions_offshore, params['map_add_region'], is_offshore=True)

### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Variable: `n.buses`

Buses related to batteries and H2 have been added.

Place `n.buses` in a dataFrame and show its content.

In [ ]:
bb = n.buses

bb.head()

How many buses are there for each carrier?

In [ ]:
bb['carrier'].value_counts()

### Variable: `n.carriers`

Carriers related to batteries and H2 have been added.

Place `n.carriers` in a dataFrame and show its content.

In [ ]:
cc = n.carriers

cc.head()

### Variable: `n.links`

Links may have been added due to:

- batteries and H2.

- interconnections with FR and PT (only for `PyPSA-Spain`)


Place `n.links` in a dataFrame and show its content.

In [ ]:
lk = n.links

lk.head()

How many links are there for each carrier?

In [ ]:
lk['carrier'].value_counts()

### Variable: `n.stores`

Stores related to batteries and H2 have been included.

Place `n.stores` in a dataFrame and show its content.

In [ ]:
st = n.stores

st.head()

How many buses are there for each carrier?

In [ ]:
st['carrier'].value_counts()

### Analysis: Interconnections (`PyPSA-Spain` only)

In `PyPSA-Spain`, interconnections with FR and PT are included at this stage.

Each interconnection is represented through a new link with a new bus. The bus is located in the border, and includes a generator and a load. The marginal cost of the generator represents the market price in FR or PT.

What are the main characteristics of DC links (interconnections and the existing ones)?

In [ ]:
lk_DC = lk.loc[lk['carrier']=='DC']

print(lk_DC[['p_nom', 'length', 'capital_cost', 'p_nom_extendable']])

What are the assumed market prices for FR and PT?

In [ ]:
### Select the prices at FR and PT
df_FR = n.generators_t['marginal_cost'].filter(like='FR').iloc[:,0].to_frame(name='EUR/MWh')
df_PT = n.generators_t['marginal_cost'].filter(like='PT').iloc[:,0].to_frame(name='EUR/MWh')

### Add 'day' and 'hour' columns
df_FR['day'] = df_PT['day'] = df_FR.index.dayofyear
df_FR['hour'] = df_PT['hour'] = df_FR.index.hour+1 



#################### Figure
fig_size = [8,6]
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=fig_size, gridspec_kw={'width_ratios': [1, 1.25]})  # ax1 is twice as wide as ax2

### Get mínimun and máximum values from 0.01 and 0.99 quantiles
vmin = min(0, df_FR['EUR/MWh'].quantile(0.01).round(), df_PT['EUR/MWh'].quantile(0.01).round())
vmax = max(df_FR['EUR/MWh'].quantile(0.99).round(), df_PT['EUR/MWh'].quantile(0.99).round())

### Make plots
fig1 = ax1.imshow(df_FR['EUR/MWh'].values.reshape(365, 24), aspect='auto', cmap='viridis', vmin=vmin, vmax=vmax)
fig2 = ax2.imshow(df_PT['EUR/MWh'].values.reshape(365, 24), aspect='auto', cmap='viridis', vmin=vmin, vmax=vmax)

ax1.set_xlabel('Hour of the Day')
ax1.set_ylabel('Day of the Year')
ax1.set_title('Hourly Prices - FR')

ax2.set_xlabel('Hour of the Day')
ax2.set_ylabel('')
ax2.set_title('Hourly Prices - PT')

plt.yticks([])  

### Add colorbar
fig.colorbar(fig2, label='Price (EUR/MWhe)')



#################### Make summary
print('FR')
print(df_FR['EUR/MWh'].describe())
print(df_FR['EUR/MWh'].quantile(0.99))

print('PT')
print(df_PT['EUR/MWh'].describe())
print(df_PT['EUR/MWh'].quantile(0.99))
